In [1]:
import pandas as pd
import yfinance as yf
from datetime import date, timedelta
from neuralprophet import NeuralProphet

In [3]:
# Gets the stock data for the last 30 days

ticker = 'MSFT'

end = date.today()
start = end - timedelta(days=360)

data = pd.DataFrame(yf.download(ticker, start, end))
data['ds'] = data.index.strftime('%m-%d-%Y')
data['y'] = data['Close'].copy()
data['volume'] = data['Volume'].copy()

# Creates the dataframe we want to use to fit

df = data.filter(['ds', 'y', 'volume'], axis=1)
df.reset_index(inplace=True)

del df['Date']

# Prints results

df.head()

[*********************100%***********************]  1 of 1 completed


,ds,y,volume
0,11-09-2021,335.950012,21307400
1,11-10-2021,330.799988,25500900
2,11-11-2021,332.429993,16849800
3,11-12-2021,336.720001,23831000
4,11-15-2021,336.070007,16723000


In [4]:
# Create and train time series model

model = NeuralProphet()

model.add_future_regressor(name='volume')

model.fit(df)

INFO - (NP.df_utils._infer_frequency) - Major frequency B corresponds to 95.984% of the data.
INFO - (NP.df_utils._infer_frequency) - Dataframe freq automatically defined as B
INFO - (NP.config.init_data_params) - Setting normalization to global as only one dataframe provided for training.
INFO - (NP.utils.set_auto_seasonalities) - Disabling yearly seasonality. Run NeuralProphet with yearly_seasonality=True to override this.
INFO - (NP.utils.set_auto_seasonalities) - Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 16
INFO - (NP.config.set_auto_batch_epoch) - Auto-set epochs to 329


  0%|          | 0/113 [00:00<?, ?it/s]

INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 5.62E-01, min: 9.60E-03


  0%|          | 0/113 [00:00<?, ?it/s]

INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 2.97E+00, min: 9.60E-03
INFO - (NP.forecaster._init_train_loader) - lr-range-test selected learning rate: 5.51E-01
Epoch[329/329]: 100%|█| 329/329 [00:10<00:00, 30.09it/s, SmoothL1Loss=0.00238, MAE=6.87, RMSE=8.44, Loss=0.0019, RegLos


,SmoothL1Loss,MAE,RMSE,Loss,RegLoss
0,0.496681,108.662175,130.729509,0.302247,0.0
1,0.381868,91.320780,112.536694,0.229866,0.0
2,0.307711,80.784335,99.193164,0.189228,0.0
3,0.257867,73.196459,89.354812,0.162099,0.0
4,0.217397,65.926305,80.963357,0.135946,0.0
...,...,...,...,...,...
324,0.002381,6.870194,8.410170,0.001899,0.0
325,0.002382,6.870932,8.420148,0.001900,0.0
326,0.002379,6.868611,8.353019,0.001897,0.0
327,0.002378,6.867937,8.388501,0.001896,0.0


In [5]:
# Gets predicitons and pirnts it to csv file

res = model.predict(df)

res.to_csv('out.csv')

# Displays last 5 trading day predictions

res.tail()

INFO - (NP.df_utils._infer_frequency) - Major frequency B corresponds to 95.984% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - B
INFO - (NP.df_utils._infer_frequency) - Major frequency B corresponds to 95.984% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - B
INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column


,ds,y,residual1,yhat1,trend,season_weekly,future_regressors_additive,future_regressor_volume
244,2022-10-28,235.869995,-10.330078,225.539917,198.591125,37.471657,-10.522873,-10.522873
245,2022-10-31,232.130005,-3.856384,228.273621,196.990128,36.755535,-5.472032,-5.472032
246,2022-11-01,228.169998,-1.199127,226.970871,196.456451,36.904942,-6.390523,-6.390523
247,2022-11-02,220.100006,4.475708,224.575714,195.922791,38.254951,-9.602039,-9.602039
248,2022-11-03,214.250000,9.191620,223.441620,195.389114,36.925861,-8.873368,-8.873368
